# Vacation Planner - LoRA Fine-tuning on Colab GPU

This notebook trains a vacation planning agent with LoRA on GPU.

**Setup:**
1. Runtime → Change runtime type → T4 GPU
2. Run all cells
3. Download trained weights at the end

## 1. Clone Repository

In [ ]:
# Clone your GitHub repo
!git clone https://github.com/vpranav5/sundial-challenge.git
%cd sundial-challenge
!ls -la

## 2. Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch transformers peft bitsandbytes accelerate sentence-transformers tqdm

# Verify GPU
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

## 3. Generate Training Data

In [ ]:
# Generate synthetic vacation planning conversations
from data_gen import VacationDataGenerator

generator = VacationDataGenerator()
generator.save_dataset("vacation_train.json", num_examples=150)
generator.save_dataset("vacation_val.json", num_examples=30)

print("\n✅ Training data generated!")

## 4. Train Model with LoRA

**Training time:** ~15-20 minutes on T4 GPU

In [ ]:
from models import VacationPlannerLLM

# Configuration
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Open model, no auth needed
lora_r = 16  # Higher rank for GPU
batch_size = 4  # Larger batch for GPU
max_steps = 100  # More steps for better quality
output_dir = "./models/vacation-planner-lora"

print("="*60)
print("VACATION PLANNER - LoRA Fine-tuning (GPU)")
print("="*60)
print(f"\nConfig:")
print(f"  Model: {model_name}")
print(f"  LoRA rank: {lora_r}")
print(f"  Steps: {max_steps}")
print(f"  Batch size: {batch_size}")
print(f"  Estimated time: 15-20 minutes\n")

# Initialize and train
llm = VacationPlannerLLM(model_name=model_name, lora_r=lora_r)
llm.load_base_model()

print("\n🚀 Starting training...\n")
llm.train(
    "vacation_train.json",
    "vacation_val.json",
    output_dir,
    batch_size=batch_size,
    max_steps=max_steps
)

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)
print(f"\nModel saved to: {output_dir}")

## 5. Test the Model

In [ ]:
# Quick test
test_prompt = "Present 3 vacation options for Bali with budget hotels and adventure activities"
response = llm.generate(test_prompt, max_length=200)
print("Test Generation:")
print(response)

## 6. Download Trained Weights

Download these files to your local machine and place them in `./models/vacation-planner-lora/`

In [ ]:
# Zip the model files
!zip -r vacation-planner-lora.zip ./models/vacation-planner-lora/

# Download link
from google.colab import files
files.download('vacation-planner-lora.zip')

print("\n✅ Download started! Extract the zip file to ./models/vacation-planner-lora/ on your local machine.")

## 7. Usage Instructions

After downloading the weights:

```bash
# On your local machine
cd ~/Desktop/sundial
unzip vacation-planner-lora.zip

# Run with fine-tuned model in demo mode
python main.py --demo --use-finetuned-llm
```